In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# Fetch your custom API key and assign it to the expected variable name
api_key = userdata.get('pinecone')
os.environ["PINECONE_API_KEY"] = api_key  # Pinecone needs this exact name

# Print for debugging (optional)
# print("Pinecone API Key:", os.environ["PINECONE_API_KEY"])


In [ ]:
from pinecone import Pinecone

# Initialize the Pinecone client
pc = Pinecone()


In [ ]:
index_name = "rag-project-piaci"

# Check if index exists before creating
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index (if it exists or was created)
index = pc.Index(index_name) # This line is correct and should work as expected

Index 'rag-project-piaci' already exists.


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.auth.credentials import AnonymousCredentials # Import AnonymousCredentials
import os

# Retrieve credentials from userdata
credentials = userdata.get('GOOGLE_API_KEY')

# Set the environment variable for authentication
os.environ['GEMINI_API_KEY'] = credentials

# Initialize GoogleGenerativeAIEmbeddings with explicit credentials
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=credentials # Pass credentials directly
)

# Embed the query
vector = embeddings.embed_query("we are building a RAG Text")

# Print or process the vector as needed
print(vector[:5])

[0.04060795158147812, -0.04385491460561752, -0.05683692544698715, -0.021194666624069214, 0.010449625551700592]


In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
    )


In [ ]:
from langchain_core.documents import Document
from uuid import uuid4
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]



In [ ]:
len(documents)

5

In [ ]:
# user_id = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents)

['9a5979f7-d725-443a-ae26-21bcf390acb6',
 '6227cd58-1127-4be9-8048-45abfd8910ae',
 'd376eda9-5596-476e-8c98-66970a2fb076',
 '710f96ba-f3eb-4f8d-ad7b-759179b4db43',
 '25646e1b-ebc3-496a-b248-180095f49ad7']

In [ ]:
# data retrive
results = vector_store.similarity_search_with_score(
    "What factors are considered when generating recommendations for users ?",)
results
# for result,score in results:
#   print(result,score)
  # print(result,similarity_search_with_score)
# results
# results = [1,[3, 5, 7, 2, 8, 10, 1]]
# highest = results[1][1]
# # highest
# for result in range(len(results)):
#   print(results[result][1])
#   if results[result][1] > highest:
#     highest = results[result][1]
# highest

#     if results[1][1] > highest:
#         highest = result
# highest


[(Document(id='1a56dbc2-e082-4b31-9e18-39c9d8be7e62', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.498711109),
 (Document(id='dd86794c-b2f6-4e85-8c7e-2c8d104c91d1', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.498711109),
 (Document(id='14bd5577-7725-4884-862c-75ba57873fd2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.498598814),
 (Document(id='128beb81-548d-4a39-baea-e1c9f0a33ed5', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.493423373)]